### Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda x,g:lambda y:g(f(y,x)), lambda x:x, lst)(x0)

In [6]:
def foldr2(f, x0, lst):
    return foldl(lambda g,x:lambda y:g(f(x,y)), lambda x:x, lst)(x0)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [7]:
def chars_range(char1,char2):
    ord_range = range(ord(char1),ord(char2))
    for number in ord_range:
        yield chr(number)

def check_inv(a, b):
    b_chars = dict.fromkeys(chars_range('a','z'),0)
    for char in b:
        b_chars[char] +=1
       
    curr_chars = dict.fromkeys(chars_range('a','z'),0)
    for char in a[:len(b)-1]:
        curr_chars[char] += 1
      
    for i in range(len(b) - 1, len(a)):
        curr_chars[a[i]] += 1
        
        if (curr_chars==b_chars):
            return True
        curr_chars[a[i-len(b)+1]] -=1
    return False



a = 'abcrotm'
b = 'tro'
check_inv(a, b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [1]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left == None and self.right == None:
            yield self.value
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right
    
    def __str__(self):
        string_repr = f'value: {str(self.value)}'
        if self.left:
            string_repr+= f' left: {str(self.left.__str__())}'
        if self.right:
            string_repr += f' right: {str(self.right.__str__())}'  
        return f'Tree: {string_repr}'
    
    def __repr__(self):
        string_repr = f'value: {str(self.value)}'
        if self.left:
            string_repr+= f' left: {str(self.left.__str__())}'
        if self.right:
            string_repr += f' right: {str(self.right.__str__())}'  
        return f'Tree: {string_repr}'
    
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [4]:
tree.__str__

<bound method Tree.__str__ of Tree: value: 0 left: Tree: value: 1 left: Tree: value: 3 right: Tree: value: 4 right: Tree: value: 2>

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [9]:
def parse(expr):
    buf = []
    for val in expr.split():
        if val[0]=='(':
            buf.append('(')
            buf.append(val[1:])
        elif val[-1]==')':
            buf.append(val[:-1])
            buf.append(')')
        else:
            buf.append(val)
    result = []
    stack = []
    high_prior = set(['*','/'])
    low_prior = set(['+','-'])
    
    for val in buf:
        if val.isnumeric():
            result.append(val)
        if val == '(':
            stack.append(val)
        if val == ')':
            while stack[-1]!='(':
                result.append(stack.pop())
            stack.pop()
        if val in low_prior:
            while len(stack) > 0 and stack[-1] in high_prior:
                result.append(stack.pop())
            stack.append(val)
        if val in high_prior:
            stack.append(val)
    
    while len(stack) > 0:
        result.append(stack.pop())
    
    return result
            

def calc(expr):
    operation = {
        '+': lambda x, y: x + y,
        '-': lambda x, y: x - y,
        '*': lambda x, y: x * y,
        '/': lambda x, y: x / y
    }
    expr = parse(expr)
    stack = []
    
    for val in expr:
        if val.isnumeric():
            stack.append(int(val))
        if val in operation:
            right_val = stack.pop()
            left_val = stack.pop()
            stack.append(operation[val](left_val,right_val))
    return stack.pop()

calc('2 * (15 - 3 * 4) - 2') == 4

True